In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Import findspark and initialize. 
import findspark

findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time
import urllib.request

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [5]:
# 2. Create a temporary view of the DataFrame.

local_filename = "home_sales_revised.csv"

# Download the file from the URL to your local machine
urllib.request.urlretrieve(url, local_filename)

# Now read the local file into a PySpark DataFrame
home_sales_df = spark.read.option("header", "true").option("inferSchema", "true").csv(local_filename)

# Now you can create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView("home_sales")

In [6]:
home_sales_df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# Spark SQL query to compute the average price of four-bedroom houses sold in each year
query = """
    SELECT 
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
"""

avg_price_df = spark.sql(query)
avg_price_df.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Spark SQL query to compute the average price of homes with 3 bedrooms and 3 bathrooms for each year they were built
query = """
    SELECT 
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""

avg_price_per_year_built_df = spark.sql(query)
avg_price_per_year_built_df.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [9]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Spark SQL query to compute the average price of homes with specified criteria for each year they were built
query = """
    SELECT 
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND 
        bathrooms = 3 AND 
        floors = 2 AND 
        sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""

avg_price_criteria_df = spark.sql(query)
avg_price_criteria_df.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Start the timer
start_time = time.time()

query = """
    SELECT 
        ROUND(AVG(price), 2) AS avg_price,
        AVG(view) AS avg_view_rating
    FROM home_sales
    WHERE price >= 350000
"""

avg_price_view_df = spark.sql(query)

# Stop the timer
end_time = time.time()

# Calculate and print the runtime
runtime = end_time - start_time
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04596853256225586 seconds ---


In [11]:
# Show the result
avg_price_view_df.show()

+---------+---------------+
|avg_price|avg_view_rating|
+---------+---------------+
|473796.26|         32.264|
+---------+---------------+



In [12]:
# 7. Cache the the temporary table home_sales.

# Cache the temporary table
spark.catalog.cacheTable("home_sales")

In [13]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")

Is 'home_sales' table cached? True


In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Start the timer
start_time = time.time()

# Define and run the query using the cached data
query = """
    SELECT 
        AVG(view) AS avg_view_rating
    FROM home_sales
    WHERE price >= 350000
"""
avg_view_rating_df = spark.sql(query)

# Stop the timer
end_time = time.time()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.011132240295410156 seconds ---


In [15]:
# Show the result
avg_view_rating_df.show()

+---------------+
|avg_view_rating|
+---------------+
|         32.264|
+---------------+



In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# The path where the partitioned Parquet files will be saved
parquet_path = "home_sales_parquet"

# Write the DataFrame to Parquet files, partitioned by the 'date_built' field
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet(parquet_path)

In [ ]:
# 11. Read the formatted parquet data.

# The path where the partitioned Parquet files are saved
parquet_path = "home_sales_parquet"

# Read the Parquet files into a DataFrame
parquet_data_df = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame to verify the data
parquet_data_df.show()

In [ ]:
# 12. Create a temporary table for the parquet data.

# Create a temporary view of the Parquet data
parquet_data_df.createOrReplaceTempView("parquet_data")

# Optionally, you can run a SQL query to verify the temporary table
query = "SELECT * FROM parquet_data LIMIT 10"
sample_data_df = spark.sql(query)
sample_data_df.show()

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

# Start the timer
start_time = time.time()

# Define and run the query on the Parquet DataFrame
query = """
    SELECT 
        ROUND(AVG(price), 2) AS avg_price,
        AVG(view) AS avg_view_rating
    FROM parquet_data
    WHERE price >= 350000
"""
avg_price_view_df_parquet = spark.sql(query)

# Stop the timer
end_time = time.time()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Show the result
avg_price_view_df_parquet.show()

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")